# !!! Run me first!!!

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
import warnings
warnings.filterwarnings('ignore')

In [2]:
#load parking ticket csv
csv_file = "Resources/Parking_Violations_DC_2018.csv"
dcparking_df = pd.read_csv(csv_file)
dcparking_df.head()

,X,Y,OBJECTID,ROWID_,DAY_OF_WEEK,HOLIDAY,WEEK_OF_YEAR,MONTH_OF_YEAR,ISSUE_TIME,VIOLATION_CODE,VIOLATION_DESCRIPTION,LOCATION,RP_PLATE_STATE,BODY_STYLE,ADDRESS_ID,STREETSEGID,XCOORD,YCOORD,TICKET_ISSUE_DATE
0,-76.9932,38.8876,4203048,NaN,NaN,NaN,NaN,NaN,NaN,P173,NO PARKING STREET CLEANING,1000 BLOCK INDEPENDENCE AVE SE S*,DC,NaN,811124,9532,400590,135528,2018-04-04T00:00:00.000Z
1,-77.0189,38.8967,4203049,NaN,NaN,NaN,NaN,NaN,NaN,P259,NO STOPPING OR STANDING IN PM RUSH HOUR ZONE,600 BLOCK 5TH ST NW EAST SIDE,VA,NaN,805251,4503,398358,136543,2018-04-04T16:08:00.000Z
2,-77.0109,38.9072,4203050,NaN,NaN,NaN,NaN,NaN,NaN,P173,NO PARKING STREET CLEANING,UNIT BLOCK N ST NW NORTH SIDE,DC,NaN,806117,5250,399058,137709,2018-04-04T10:04:00.000Z
3,-77.0786,38.9127,4203051,NaN,NaN,NaN,NaN,NaN,NaN,P198,PARK WITHIN 20 FEET OF A BUS STOP OR ZONE,3900 BLOCK RESERVOIR RD NW SOUTH*,MD,NaN,812695,10916,393186,138317,2018-04-04T16:31:00.000Z
4,-77.0418,38.9233,4203052,NaN,NaN,NaN,NaN,NaN,NaN,P281,FAIL TO DISPLAY A MULTISPACE METER RECEIPT,1700 BLOCK COLUMBIA RD NW SOUTH *,MA,NaN,800405,339,396379,139491,2018-04-04T19:22:00.000Z


In [3]:
#rename columns from  X,Y to Longitude, Latitude
dcparking_df.rename(columns = {'X': 'Longitude' , 'Y': 'Latitude'}, inplace=True)

In [4]:
#list the columns
dcparking_df.columns

Index(['Longitude', 'Latitude', 'OBJECTID', 'ROWID_', 'DAY_OF_WEEK', 'HOLIDAY',
       'WEEK_OF_YEAR', 'MONTH_OF_YEAR', 'ISSUE_TIME', 'VIOLATION_CODE',
       'VIOLATION_DESCRIPTION', 'LOCATION', 'RP_PLATE_STATE', 'BODY_STYLE',
       'ADDRESS_ID', 'STREETSEGID', 'XCOORD', 'YCOORD', 'TICKET_ISSUE_DATE'],
      dtype='object')

In [5]:
#set the new columns we want to use.
newdcpark_df = dcparking_df[['Latitude', 'Longitude', 'VIOLATION_DESCRIPTION','TICKET_ISSUE_DATE' ]]
newdcpark_df.head()

,Latitude,Longitude,VIOLATION_DESCRIPTION,TICKET_ISSUE_DATE
0,38.8876,-76.9932,NO PARKING STREET CLEANING,2018-04-04T00:00:00.000Z
1,38.8967,-77.0189,NO STOPPING OR STANDING IN PM RUSH HOUR ZONE,2018-04-04T16:08:00.000Z
2,38.9072,-77.0109,NO PARKING STREET CLEANING,2018-04-04T10:04:00.000Z
3,38.9127,-77.0786,PARK WITHIN 20 FEET OF A BUS STOP OR ZONE,2018-04-04T16:31:00.000Z
4,38.9233,-77.0418,FAIL TO DISPLAY A MULTISPACE METER RECEIPT,2018-04-04T19:22:00.000Z


## Getting date information

In [6]:
# clean up date to make it just month and day
dates = []
dates = newdcpark_df['TICKET_ISSUE_DATE'].values
dates [1]

'2018-04-04T16:08:00.000Z'

In [7]:
# get the month

MM = []
for d in dates:
    MM.append(d[5:7])
print(MM)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
#get the day
DD = []
for d in dates:
    DD.append(d[8:10])
print(DD)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
#create the MONTH column and populate (you'll get an error but it works)
newdcpark_df["MONTH"]=MM

In [10]:
#create the DAY column and populate (you'll get an error but it works)
newdcpark_df["DAY"]=DD

In [11]:
#put date back together in YYYY-MM-DD format
newdcpark_df['DATE']=[f"2018-{newdcpark_df['MONTH'][i]}-{newdcpark_df['DAY'][i]}" for i in newdcpark_df.index]


In [12]:
#create new columns (getting rid of the ticket_issue_date)
finaldcpark_df = newdcpark_df[['Latitude', 'Longitude', 'VIOLATION_DESCRIPTION', 'MONTH', 'DAY', 'DATE']]
finaldcpark_df.head()

,Latitude,Longitude,VIOLATION_DESCRIPTION,MONTH,DAY,DATE
0,38.8876,-76.9932,NO PARKING STREET CLEANING,04,04,2018-04-04
1,38.8967,-77.0189,NO STOPPING OR STANDING IN PM RUSH HOUR ZONE,04,04,2018-04-04
2,38.9072,-77.0109,NO PARKING STREET CLEANING,04,04,2018-04-04
3,38.9127,-77.0786,PARK WITHIN 20 FEET OF A BUS STOP OR ZONE,04,04,2018-04-04
4,38.9233,-77.0418,FAIL TO DISPLAY A MULTISPACE METER RECEIPT,04,04,2018-04-04


## Push to SQL

In [13]:
#Create Database
connection_string = "root:Password@localhost/DCDATA_DB?charset=utf8"
engine = create_engine(f'mysql://{connection_string}')

In [14]:
#load new data after dropping columns from Parking
newdcpark_df.to_sql(name='PARKING', con=engine, if_exists='replace', index=False)

In [15]:
#confirm new parking data loaded successfully
pd.read_sql_query('select * from PARKING', con=engine).head(5)

,Latitude,Longitude,VIOLATION_DESCRIPTION,TICKET_ISSUE_DATE,MONTH,DAY,DATE
0,38.88760034879,-76.993201829703,NO PARKING STREET CLEANING,2018-04-04T00:00:00.000Z,04,04,2018-04-04
1,38.8967425107048,-77.0189307155777,NO STOPPING OR STANDING IN PM RUSH HOUR ZONE,2018-04-04T16:08:00.000Z,04,04,2018-04-04
2,38.9072472840597,-77.0108629472506,NO PARKING STREET CLEANING,2018-04-04T10:04:00.000Z,04,04,2018-04-04
3,38.9126984450469,-77.07856940182,PARK WITHIN 20 FEET OF A BUS STOP OR ZONE,2018-04-04T16:31:00.000Z,04,04,2018-04-04
4,38.9232931808667,-77.0417595163018,FAIL TO DISPLAY A MULTISPACE METER RECEIPT,2018-04-04T19:22:00.000Z,04,04,2018-04-04
